## Import library

In [1]:
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 

from kan import *
import warnings
import sys
sys.path.append('../utils')
from treasury_base import *

warnings.filterwarnings("ignore")

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Retrieve data

In [4]:
WINDOW_LIST = [5, 10]
LAG = 1

def train_mse():
    predictions = model(dataset['train_input'])  # Model predictions
    mse = F.mse_loss(predictions, dataset['train_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5  # Return scalar MSE value

def test_mse():
    predictions = model(dataset['test_input']) # Model predictions
    mse = F.mse_loss(predictions, dataset['test_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5
    
df_ma = ma_data_retrieval(window_list=WINDOW_LIST, lag=LAG)
df_ma.head()

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,...,5 Yr_MA5,5 Yr_MA10,7 Yr_MA5,7 Yr_MA10,10 Yr_MA5,10 Yr_MA10,20 Yr_MA5,20 Yr_MA10,30 Yr_MA5,30 Yr_MA10
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-16,2.41,2.40,2.43,2.49,2.57,2.55,2.53,2.54,2.62,2.73,...,2.542,2.517,2.616,2.587,2.724,2.694,2.910,2.870,3.054,3.013
2019-01-17,2.41,2.41,2.42,2.50,2.57,2.56,2.55,2.58,2.66,2.75,...,2.536,2.522,2.612,2.593,2.722,2.701,2.914,2.879,3.062,3.023
2019-01-18,2.40,2.40,2.41,2.50,2.60,2.62,2.60,2.62,2.70,2.79,...,2.540,2.543,2.618,2.615,2.724,2.720,2.916,2.897,3.064,3.038
2019-01-22,2.38,2.40,2.43,2.51,2.59,2.58,2.55,2.57,2.65,2.74,...,2.560,2.556,2.638,2.629,2.740,2.732,2.926,2.909,3.074,3.049
2019-01-23,2.37,2.38,2.41,2.51,2.59,2.58,2.57,2.59,2.66,2.76,...,2.568,2.560,2.648,2.634,2.746,2.736,2.926,2.914,3.074,3.056


## KAN model training

In [ ]:
TEST_SIZE = 1
LENGTH = len(df_ma)
TARGETS = df_ma.columns[:12]

# Store results for each fold
fold_results = {'train_mse': [], 'test_mse': [], 'naive_mse': []}

for cnt in range(0, 20, 5):
    print()
    print('WINDOW SLIDING: ', cnt)

    df_window = df_ma[(LENGTH-cnt-120):(LENGTH-cnt)]
    # Prepare data
    X, y = df_window.drop(columns=TARGETS), df_window[TARGETS]

    # scaler = StandardScaler()
    # X = pd.DataFrame(scaler.fit_transform(X))

    n_inputs = X.shape[1]
    n_outputs = y.shape[1]

    X_train, X_test = X[:-TEST_SIZE], X[-TEST_SIZE:]
    y_train, y_test = y[:-TEST_SIZE], y[-TEST_SIZE:]

    dataset = dict()
    dtype = torch.get_default_dtype()
    dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
    dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
    dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
    dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)

    # Initialize the model
    model = KAN(width=[n_inputs, 32, n_outputs], grid=3, k=2, seed=42, device=device)

    # Train the model and compute metrics
    results = model.fit(dataset, opt="Adam", lamb=0.001, lr=0.001, steps=1500, metrics=(train_mse, test_mse))
    df_naive = pd.DataFrame([y_train.iloc[-1]] * TEST_SIZE, columns=y_train.columns)
        
    # Store the metrics
    train_error = results['train_mse'][-1]
    test_error = results['test_mse'][-1]
    naive_error = mean_squared_error(df_naive, y_test, squared=False)

    fold_results['train_mse'].append(train_error)
    fold_results['test_mse'].append(test_error)
    fold_results['naive_mse'].append(naive_error)

    # Calculate average metrics across all windows
    print(f'Fold Train MSE: {train_error}')
    print(f'Fold Test MSE: {test_error}')
    print(f'Naive Test MSE: {naive_error}')

avg_train_mse = np.mean(fold_results['train_mse'])
avg_test_mse = np.mean(fold_results['test_mse'])
avg_naive_mse = np.mean(fold_results['naive_mse'])

print()
print("Sliding Window Cross-Validation Results")
print(f"Average Train MSE: {avg_train_mse}")
print(f"Average Test MSE: {avg_test_mse}")
print(f"Average Naive MSE: {avg_naive_mse}")

## Optuna training

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

In [ ]:
import optuna
import torch

# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space
    n_layers = trial.suggest_int('n_layers', 1, 2)  # Number of layers in the network
    layer_sizes = [trial.suggest_int(f'n_units_l{i}', 16, 64, step=16) for i in range(n_layers)]
    grid = trial.suggest_int('grid', 2, 4)          # Example parameter for KAN
    lamb = trial.suggest_float('lamb', 1e-4, 1e-2, log=True)  # Regularization rate
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)       # Learning rate
    steps = trial.suggest_int('steps', 500, 2000, step=500)   # Training steps

    # Model architecture
    width = [n_inputs] + layer_sizes + [n_outputs]

    # Initialize dataset
    dataset = dict()
    dtype = torch.get_default_dtype()
    dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
    dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
    dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
    dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)

    # Initialize the model
    model = KAN(width=width, grid=grid, k=2, seed=42, device=device)

    # Train the model
    results = model.fit(
        dataset, 
        opt="Adam", 
        lamb=lamb, 
        lr=lr, 
        steps=steps, 
        metrics=(train_mse, test_mse)
    )
    
    # Retrieve the metric (e.g., test MSE) from the results
    test_mse_value = results['test_mse']
    return test_mse_value  # Minimize test MSE

# Create an Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Best parameters and results
print("Best parameters:", study.best_params)
print("Best test MSE:", study.best_value)
